## Queries - ComCam

In this notebook, we show how to query the ComCam repository\
and view the resulting images.\
Craig Lage - 13-Jul-24

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as pf
from lsst.daf.butler import Butler
from lsst.ip.isr import IsrTask, IsrTaskConfig
from lsst.summit.utils.plotting import plot
import lsst.afw.cameraGeom.utils as camGeomUtils

In [ ]:
butler = Butler('/repo/LSSTComCam', collections=["LSSTComCam/raw/all","LSSTComCam/calib"])

## First, get a list of exposures
### These should match what you see in RubinTV.

In [ ]:
dayObs = 20241024
exposureList = []
for record in butler.registry.queryDimensionRecords("exposure", where="exposure.day_obs=%d"%dayObs):
    exposureList.append([record.id, record])
exposureList.sort(key=lambda x: x[0])
for i, [id,record] in enumerate(exposureList):
    print(record.id, record.observation_type, record.exposure_time, record.physical_filter)
    if i > 0 and (exposureList[i][0] - exposureList[i-1][0]) != 1:
        print("Missed one!")

# Get the data from the headers

In [ ]:
expId = 7024101500005 
for detector in range(9):
    raw = butler.get('raw', detector=detector, exposure=expId)
    #mData = butler.get('raw.metadata', detector=4, exposure=expId)
    #for key in mData.keys():
    #    print(key, mData[key])

## Define a simple ISR
### Just overscan subtraction and bias subtraction.

In [ ]:
isrConfig = IsrTaskConfig()
isrConfig.doLinearize=False
isrConfig.doOverscan=True
isrConfig.overscan.fitType="MEDIAN_PER_ROW"
isrConfig.overscan.doParallelOverscan=True
isrConfig.doAssembleCcd=True
isrConfig.doBias=True
isrConfig.doVariance=False
isrConfig.doCrosstalk=False
isrConfig.doBrighterFatter=False
isrConfig.doDark=False
isrConfig.doStrayLight=False
isrConfig.doFlat=False
isrConfig.doFringe=False
isrConfig.doApplyGains=True
isrConfig.usePtcGains=True
isrConfig.doDefect=False
isrConfig.doNanMasking=True
isrConfig.doInterpolate=False
isrConfig.doSaturation=False
isrConfig.doSaturationInterpolation=False
isrTask = IsrTask(config=isrConfig)

## Run the ISR and look at the result
### Here I've taken an exposure where I found a cosmic ray using the RUbinTV CCS image viewer image

In [ ]:
expId = 2024102400089
exp = butler.get('raw', detector=4, exposure=expId)
biasExp = butler.get('bias', detector=4, exposure=expId) # This is a bias image associated with the data
ptc = butler.get('ptc', detector=4)
#ct = butler.get('crosstalk', detector=4)
isrResult = isrTask.run(exp, bias=biasExp, ptc=ptc) # This runs the ISR

In [ ]:
%matplotlib inline
x = plot(isrResult.exposure, stretch='ccs')
plt.savefig(f"/home/cslage/DATA/ComCam_{expId}.png")

# Plot a small region

In [ ]:
x = plot(isrResult.exposure.image.array[100:200, 1000:1100])

In [ ]:
plt.title(f"ComCam {expId}, Bias")
plt.plot(isrResult.exposure.image.array[2100, :], label='X cut')
plt.plot(isrResult.exposure.image.array[:, 2100], label='Y cut')
#plt.ylim(0,70000)
plt.ylim(-50, 50)
plt.ylabel("Flux (electrons)")
plt.xlabel("Pixels")
plt.legend()
plt.savefig(f"/home/cslage/DATA/ComCam_Slices_{expId}.png")

# Now assemble all 9 CCDs and plot the result

In [ ]:
def myCallback(im, ccd, imageSource):
    """Assemble the CCD image and do basic ISR"""
    dayObs = imageSource.kwargs['day_obs']
    seqNum = imageSource.kwargs['seq_num']
    exp = imageSource.butler.get('raw', detector=ccd.getId(), day_obs=dayObs, seq_num=seqNum)
    biasExp = imageSource.butler.get('bias', detector=ccd.getId(), day_obs=dayObs, seq_num=seqNum)
    isrResult = isrTask.run(exp, bias=biasExp)
    oim = isrResult.exposure.image
    return oim

In [ ]:
%matplotlib inline
instrument = "LSSTComCam"
camera = butler.get('camera', instrument=instrument)
fig = plt.figure(figsize=(12,12))
import lsst.afw.display as afwDisplay
disp = afwDisplay.Display(1, "matplotlib")
#disp.scale('linear', min = 45000, max=65000)
disp.scale('linear', min='zscale')
dayObs = 20240729
seqNum = 257
dataType='raw'
mos = camGeomUtils.showCamera(camera,
                              camGeomUtils.ButlerImage(butler, dataType, 
                                                       instrument=instrument, raft="R22",
                                                       day_obs=dayObs, seq_num=seqNum,
                                                       verbose=False, callback=myCallback,
                                                       background=np.nan),
                              binSize=1, display=disp, overlay=False,
                              title="%d %d" % (dayObs, seqNum))

plt.savefig(f"/home/cslage/DATA/ComCam_{dayObs}_{seqNum}.png")

In [ ]:
%matplotlib inline
plt.title(f"ComCam {dayObs}_{seqNum}")
plt.plot(mos.array[6000, :], label="X slice")
plt.plot(mos.array[:, 6000], label="Y slice")
#plt.ylim(0, 50000)
plt.ylim(-50, 50)
plt.ylabel("Flux (electrons)")
plt.xlabel("Pixels")
plt.legend()
plt.savefig(f"/home/cslage/DATA/ComCam_Slices_{dayObs}_{seqNum}.png")

In [ ]:
plt.hist(mos.array.flatten(), range=(-50,50))
plt.text(20,4E7, f"Sigma = {np.nanstd(mos.array.flatten()):.2f} e-")


# Plot a small region

In [ ]:
x = plot(mos.array[3900:4000, 4500:4600])

# The cell below will save the combined image as a FITS file

In [ ]:
filename = "/home/cslage/DATA/FITS_2024071200007.fits"
mos.writeFits(filename)

# Variability per amp

In [ ]:
%matplotlib inline
clip = 10.0
expId = 2024071200007

for det in range(9):
    exp = butler.get('raw', detector=det, exposure=expId)
    biasExp = butler.get('bias', detector=det, exposure=expId) # This is a bias image associated with the data
    isrResult = isrTask.run(exp, bias=biasExp) # This runs the ISR
    for amp in exp.getDetector().getAmplifiers():
        data = isrResult.exposure[amp.getBBox()].image.array.flatten()
        data = (data - np.median(data)) / np.median(data) * 100.0
        sortedData = np.sort(data)
        sortedTrimmedData = np.clip(sortedData, np.percentile(sortedData, clip), np.percentile(sortedData, 100 - clip))
        plt.plot(sortedTrimmedData)
plt.ylim(-5.0, 5.0)
plt.title(f"ComCam {expId}, per amp variability - 10->90 percentiles")
plt.xlabel("Pixel count")
plt.ylabel("Percent variation from Median")
plt.savefig(f"/home/cslage/DATA/ComCam_Amp_Nonuniformity_{expId}.png")